Reproducing code from https://github.com/inhuh/trs-oden/blob/main/run_exp1.py

In [1]:
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Lambda, Layer
from keras.layers import Dense
from keras.layers import Add, Concatenate
from keras.optimizers import Adam

In [2]:
class Gradient(Layer):
    def call(self, inputs, **kwargs):
        x, y = inputs
        return K.gradients(loss=y, variables=x)[0]

    def compute_output_shape(self, input_shape):
        return input_shape[0]


def time_difference(x):
    return x[:, 1:] - x[:, :-1]


def reversing_operator(x, dim, with_batch):
    if with_batch:
        q = x[:, :, :dim]
        p = -x[:, :, dim:]
        return K.concatenate(tensors=[q, p], axis=-1)
    else:
        q = x[:, :dim]
        p = -x[:, dim:]
        return K.concatenate(tensors=[q, p], axis=-1)


class ODENetwork:
    def __init__(self, nb_object=1, nb_coords=1, function_type='ode', time_horizon=10, time_augment=False,
                 nb_units=1000, nb_layers=1, activation='tanh',
                 lambda_trs=0.0, use_time_dep_lambda=False, learning_rate=2e-4):
        self.dim = int(nb_object * nb_coords)
        self.T = time_horizon
        self.augment = time_augment
        self.units, self.layers = nb_units, nb_layers
        self.act = activation
        self.lambda_trs = lambda_trs
        self.t_dep = use_time_dep_lambda
        self.lr = learning_rate
        if function_type == 'ode':
            self.func = self.ode_function()
        elif function_type == 'hamiltonian':
            self.func = self.hamilton_equation()
        else:
            raise NotImplementedError

    def ode_function(self):
        x = Input(shape=(int(2 * self.dim),))
        if self.augment:
            t = Input(shape=(1,))
            h = Dense(units=self.units, activation=self.act)(Concatenate(axis=-1)([x, t]))
        else:
            h = Dense(units=self.units, activation=self.act)(x)
        for _ in range((self.layers - 1)):
            h = Dense(units=self.units, activation=self.act)(h)
        y = Dense(units=int(2 * self.dim), use_bias=False)(h)
        if self.augment:
            return Model(inputs=[t, x], outputs=y)
        else:
            return Model(inputs=x, outputs=y)

    def hamilton_equation(self):
        x = Input(shape=(int(2 * self.dim),))
        q, p = Lambda(lambda f: f[:, :self.dim])(x), Lambda(lambda f: f[:, self.dim:])(x)
        if self.augment:
            t = Input(shape=(1,))
            v = Dense(units=self.units, activation=self.act)(Concatenate(axis=-1)([q, t]))
            k = Dense(units=self.units, activation=self.act)(Concatenate(axis=-1)([p, t]))
        else:
            v = Dense(units=self.units, activation=self.act)(q)
            k = Dense(units=self.units, activation=self.act)(p)
        for _ in range((self.layers - 1)):
            v = Dense(units=self.units, activation=self.act)(v)
            k = Dense(units=self.units, activation=self.act)(k)
        v = Dense(units=1, use_bias=False)(v)
        k = Dense(units=1, use_bias=False)(k)
        dq = Gradient()([p, k])
        dp = Lambda(lambda f: -1 * f)(Gradient()([q, v]))
        if self.augment:
            return Model(inputs=[t, x], outputs=Concatenate(axis=-1)([dq, dp]))
        else:
            return Model(inputs=x, outputs=Concatenate(axis=-1)([dq, dp]))

    def solver(self):
        def l_ode(y_true, y_pred):
            return K.mean(K.square(y_true - y_pred))

        def l_trs(y_true, y_pred):
            if self.t_dep:
                norm_ts = ts / K.max(ts)
                return K.mean(norm_ts[:, 1:, 0] * K.mean(K.square(reversing_operator(x=X, dim=self.dim, with_batch=True) - Xr), axis=-1))
            else:
                return K.mean(K.square(reversing_operator(x=X, dim=self.dim, with_batch=True) - Xr))

        def l_trsoden(y_true, y_pred):
            return l_ode(y_true, y_pred) + self.lambda_trs * l_trs(y_true, y_pred)

        ts = Input(shape=(self.T + 1, 1))
        dts = Lambda(function=time_difference)(ts)

        x0 = Input(shape=(int(2 * self.dim),))
        x = x0
        xr = Lambda(function=reversing_operator, arguments={'dim': self.dim, 'with_batch': False})(x)

        ls_x, ls_xr = [], []

        for i in range(self.T):
            t = Lambda(lambda f: f[:, i, :])(ts)
            tr = Lambda(lambda f: -f)(t)
            dt = Lambda(lambda f: f[:, i, :])(dts)

            if self.augment:  # For time-augmented (non-autonomous) cases, we used Runge-Kutta 4th solver.
                # Forward time evolution
                dx1 = Lambda(lambda f: f*dt)(self.func([t, x]))
                dx2 = Lambda(lambda f: f*dt)(self.func([Add()([t, Lambda(lambda f: 0.5*f)(dt)]), Add()([x, Lambda(lambda f: 0.5*f)(dx1)])]))
                dx3 = Lambda(lambda f: f*dt)(self.func([Add()([t, Lambda(lambda f: 0.5*f)(dt)]), Add()([x, Lambda(lambda f: 0.5*f)(dx2)])]))
                dx4 = Lambda(lambda f: f*dt)(self.func([Add()([t, dt]), Add()([x, dx3])]))
                dx = Lambda(lambda f: (1/6)*f)(Add()([dx1, Lambda(lambda f: 2*f)(dx2), Lambda(lambda f: 2*f)(dx3), dx4]))
                x = Add()([x, dx])
                ls_x.append(x)

                # Backward time evolution
                dxr1 = Lambda(lambda f: -f*dt)(self.func([tr, xr]))
                dxr2 = Lambda(lambda f: -f*dt)(self.func([Add()([tr, Lambda(lambda f: -0.5*f)(dt)]), Add()([xr, Lambda(lambda f: 0.5*f)(dxr1)])]))
                dxr3 = Lambda(lambda f: -f*dt)(self.func([Add()([tr, Lambda(lambda f: -0.5*f)(dt)]), Add()([xr, Lambda(lambda f: 0.5*f)(dxr2)])]))
                dxr4 = Lambda(lambda f: -f*dt)(self.func([Add()([tr, Lambda(lambda f: -f)(dt)]), Add()([xr, dxr3])]))
                dxr = Lambda(lambda f: (1/6)*f)(Add()([dxr1, Lambda(lambda f: 2*f)(dxr2), Lambda(lambda f: 2*f)(dxr3), dxr4]))
                xr = Add()([xr, dxr])
                ls_xr.append(xr)

            else:  # Leapfrog solver for autonomous systems
                # Forward time evolution
                q, p = Lambda(lambda f: f[:, :self.dim])(x), Lambda(lambda f: f[:, self.dim:])(x)
                p = Add()([p, Lambda(lambda f: (0.5*f*dt)[:, self.dim:])(self.func(x))])
                q = Add()([q, Lambda(lambda f: (1.0*f*dt)[:, :self.dim])(self.func(Concatenate(axis=-1)([q, p])))])
                p = Add()([p, Lambda(lambda f: (0.5*f*dt)[:, self.dim:])(self.func(Concatenate(axis=-1)([q, p])))])
                x = Concatenate(axis=-1)([q, p])
                ls_x.append(x)

                # Backward time evolution
                qr, pr = Lambda(lambda f: f[:, :self.dim])(xr), Lambda(lambda f: f[:, self.dim:])(xr)
                pr = Add()([pr, Lambda(lambda f: (-0.5*f*dt)[:, self.dim:])(self.func(xr))])
                qr = Add()([qr, Lambda(lambda f: (-1.0*f*dt)[:, :self.dim])(self.func(Concatenate(axis=-1)([qr, pr])))])
                pr = Add()([pr, Lambda(lambda f: (-0.5*f*dt)[:, self.dim:])(self.func(Concatenate(axis=-1)([qr, pr])))])
                xr = Concatenate(axis=-1)([qr, pr])
                ls_xr.append(xr)

        X = Lambda(lambda f: K.stack(f, axis=1))(ls_x)
        Xr = Lambda(lambda f: K.stack(f, axis=1))(ls_xr)

        model = Model(inputs=[ts, x0], outputs=X)
        model.compile(optimizer=Adam(lr=self.lr), loss=l_trsoden, metrics=[l_ode, l_trs])
        return model

In [3]:
from data_duffing import get_dataset
from utils import leapfrog_solver, runge_kutta_solver, reshape_data
import matplotlib.pyplot as plt
import os
import argparse
import numpy as np
import pickle as pkl

In [4]:
%matplotlib inline

In [5]:
this_dir = os.path.abspath(os.getcwd())
this_dir

'/home/ajivani/generative_experiments/trs-oden'

In [6]:
%matplotlib inline

In [7]:
def get_args():
    parser = argparse.ArgumentParser(description=None)
    parser.add_argument('--save_folder', default='experiment-duffing-1', type=str)

    # Data variables
    parser.add_argument('--osc_params', default=(1, 0, 0, 0), type=tuple)
    parser.add_argument('--train_nb_sample', default=50, type=int)
    parser.add_argument('--train_nb_timestep', default=30, type=int)
    parser.add_argument('--train_t_span', default=(0, 3), type=tuple)
    parser.add_argument('--train_seed', default=0, type=int)
    parser.add_argument('--train_noise_level', default=0.1, type=float)
    parser.add_argument('--test_nb_sample', default=50, type=int)
    parser.add_argument('--test_nb_timestep', default=200, type=int)
    parser.add_argument('--test_t_span', default=(0, 20), type=tuple)
    parser.add_argument('--test_seed', default=999, type=int)
    parser.add_argument('--test_noise_level', default=0.0, type=float)

    # Model variables
    parser.add_argument('--nb_object', default=1, type=int)
    parser.add_argument('--nb_coords', default=1, type=int)
    parser.add_argument('--time_horizon', default=10, type=int)
    parser.add_argument('--time_augment', action='store_true')
    parser.add_argument('--nb_units', default=1000, type=int)
    parser.add_argument('--nb_layers', default=1, type=int)
    parser.add_argument('--activation', default='tanh', type=str)
    parser.add_argument('--use_time_dep_lambda', action='store_true')
    parser.add_argument('--ls_cond', default=[['ode', 0.0], ['hamiltonian', 0.0], ['ode', 10.0]])
    parser.add_argument('--ls_color', default=['red', 'dodgerblue', 'seagreen'])
    parser.add_argument('--learning_rate', default=2e-4, type=float)
    parser.add_argument('--epochs', default=5000, type=int)
    parser.add_argument('--verbose', default=0, type=int)

    return parser.parse_args(args=())

In [8]:
args = get_args()

In [9]:
args.save_folder

'experiment-duffing-1'

In [10]:
def run(args):
    plt.rc('font', size=10)
    plt.rc('axes', labelsize=12)
    plt.rcParams['lines.linewidth'] = 3
    plt.rcParams['figure.figsize'] = 4.5, 4

    save_dir = os.path.join(this_dir, args.save_folder)
    os.makedirs(save_dir, exist_ok=True)
    print('save directory:', save_dir)

    for i in range(len(args.ls_cond)):
        cond, col = args.ls_cond[i], args.ls_color[i]
        function_type, lambda_trs = cond[0], cond[1]
        split_name = str(function_type) + '_' + str(lambda_trs) + '_' + str(args.use_time_dep_lambda)
        print('experimental condition:', split_name)

        ts, xs = get_dataset(nb_samples=args.train_nb_sample, nb_timestep=args.train_nb_timestep,
                             t_span=args.train_t_span, seed=args.train_seed, noise_level=args.train_noise_level,
                             params=args.osc_params)
        t_train, x_train, y_train = reshape_data(ts=ts, xs=xs, substep=args.time_horizon)

        oden = ODENetwork(nb_object=args.nb_object, nb_coords=args.nb_coords, function_type=function_type,
                          time_horizon=args.time_horizon, time_augment=args.time_augment,
                          nb_units=args.nb_units, nb_layers=args.nb_layers,
                          activation=args.activation, lambda_trs=lambda_trs,
                          use_time_dep_lambda=args.use_time_dep_lambda, learning_rate=args.learning_rate)

        oden.solver().fit(x=[t_train, x_train], y=y_train, epochs=args.epochs, batch_size=len(x_train), verbose=args.verbose)
        oden.func.save_weights(os.path.join(save_dir, split_name + '_' + 'weight.h5'))

        ts, xs = get_dataset(nb_samples=args.test_nb_sample, nb_timestep=args.test_nb_timestep,
                             t_span=args.test_t_span, seed=args.test_seed, noise_level=args.test_noise_level,
                             params=args.osc_params)
        t_test, x_test, y_test = ts, xs[:, 0, :], xs[:, 1:, :]

        if args.time_augment:
            y_pred = runge_kutta_solver(ts=t_test, x0=x_test, func=oden.func)
        else:
            y_pred = leapfrog_solver(ts=t_test, x0=x_test, func=oden.func, dim=int(args.nb_object * args.nb_coords))

        q_true, p_true, q_pred, p_pred = y_test[:, :, 0], y_test[:, :, 1], y_pred[:, :, 0], y_pred[:, :, 1]
        e_true = args.osc_params[0] * np.square(q_true) + (args.osc_params[1] * np.square(np.square(q_true)) / 2) + np.square(p_true)
        e_pred = args.osc_params[0] * np.square(q_pred) + (args.osc_params[1] * np.square(np.square(q_pred)) / 2) + np.square(p_pred)

        mse_t_per_sample = np.mean(np.mean(np.square(y_test - y_pred), axis=-1), axis=-1)
        mse_t, std_t = 1e2 * np.mean(mse_t_per_sample), 1e2 * np.std(mse_t_per_sample)

        mse_e_per_sample = np.mean(np.square(e_true - e_pred), axis=-1)
        mse_e, std_e = 1e2 * np.mean(mse_e_per_sample), 1e2 * np.std(mse_e_per_sample)
        print('trajectory MSE: {:.2f} pm {:.2f}'.format(mse_t, std_t))
        print('energy MSE: {:.2f} pm {:.2f}'.format(mse_e, std_e))

        result = {'ground_truth': y_test, 'predicted': y_pred, 'mse_t': mse_t, 'std_t': std_t, 'mse_e': mse_e, std_t: std_e}

        with open(os.path.join(save_dir, split_name + '_' + 'result.pkl'), 'wb') as f:
            pkl.dump(result, f)

        plt.plot(q_true[1], p_true[1], c='k', label='Ground truth')
        plt.plot(q_pred[1], p_pred[1], c=col, label=split_name)
        plt.grid(), plt.legend()
        plt.xlabel('Position q'), plt.ylabel('Momentum p')
        plt.xlim([-1.2, 1.2]), plt.ylim([-1.2, 1.2])
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, split_name + '_' + 'trajectory.png'))
        plt.close()

        plt.plot(t_test[0, 1:].squeeze(), e_true[1], c='k', label='Ground truth')
        plt.plot(t_test[0, 1:].squeeze(), e_pred[1], c=col, label=split_name)
        plt.grid(), plt.legend()
        plt.xlabel('Time'), plt.ylabel('Total energy')
        plt.xlim([0, args.test_t_span[1]]), plt.ylim([0, 1.2])
        plt.tight_layout()
        plt.savefig(os.path.join(save_dir, split_name + '_' + 'energy.png'))
        plt.close()

In [11]:
run(args)

save directory: /home/ajivani/generative_experiments/trs-oden/experiment-duffing-1
experimental condition: ode_0.0_False


NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 2), dtype=float32, sparse=None, name=keras_tensor_7>',)
  • kwargs={'mask': 'None'}